In [ ]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
llm = ChatOllama(model="llama3.2:1b")

def chat_node(state: ChatState):

  messages = state['messages']

  response = llm.invoke(messages)

  return {'messages':[response]}

In [ ]:
graph = StateGraph(ChatState)

checkpointer = InMemorySaver()

graph.add_node(chat_node, 'chat_node')
graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)


In [ ]:
chatbot

In [ ]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of pakistan?')]
}

chatbot.invoke(initial_state)['messages'][-1].content

In [ ]:
thread_id = '1'
while True:
    user_input = input("You: ")
    print("User:", user_input)

    if user_input.lower() in ['exit', 'quit','bye']:
        break
    config ={'configurable':{'thread_id':thread_id}}
    
    response = chatbot.invoke({'messages': [HumanMessage(content=user_input)]
    },config=config)
    print("AI:", response['messages'][-1].content)